In [2]:
import findspark
findspark.init()
from pyspark.mllib.recommendation import ALS, Rating
from pyspark import SparkContext
sc = SparkContext("local", "test")

In [6]:
# Load and parse the data
rating_data = sc.textFile("./data/ml-10M100K/ratings-train.dat")
ratings_train = (rating_data.map(lambda l: l.split(','))
    .map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))
           )

In [11]:
validation_data = sc.textFile("./data/ml-10M100K/ratings-validation.dat")
ratings_validation = (rating_data.map(lambda l: l.split(','))
    .map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))
           )

#### test.first()

In [7]:
ratings_train.take(3)

[Rating(user=36955, product=21, rating=3.0),
 Rating(user=36955, product=47, rating=5.0),
 Rating(user=36955, product=1079, rating=3.0)]

In [9]:
# Build the recommendation model using Alternating Least Squares
#rank is number of latent variable we want to set
rank = 10
numIterations = 10
lambda_value = 0.01
model = ALS.train(ratings_train, rank, numIterations, lambda_value)

In [12]:
# Evaluate the model on training data
testdata = ratings_validation.map(lambda p: (p[0], p[1]))
predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = ratings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error = " + str(MSE))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 191.0 failed 1 times, most recent failure: Lost task 0.0 in stage 191.0 (TID 87, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-3-9dbbec2ea347>", line 4, in <lambda>
ValueError: invalid literal for int() with base 10: '36955,21,3.0,789652009'

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:166)
	at org.apache.spar